# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096927


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.31s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:43,  1.68s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.27s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:23,  1.00it/s]

...

Rendering models:  73%|███████▎  | 22/30 [00:11<00:01,  4.88it/s]

Rendering models:  80%|████████  | 24/30 [00:11<00:01,  5.82it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  7.49it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  6.89it/s]

not-logged-in-0687f3923f81ac7bd40a    0.007564
Jnursssmith5263                       0.034599
mwalsh868                             0.000993
Quarkgc                               0.000691
jnarayanbvg                           0.000978
ellivc                                0.000816
not-logged-in-c6917d9b19f41e6c2995    0.001409
not-logged-in-d78ab5a3057206e8280b    3.210693
framosuw                              0.001076
lsautter                              0.000917
Saharisunshine                        0.001182
RaulE                                 0.000589
acapirala                             0.001016
pangeli5                              0.000653
farheenk88                            0.000538
Lavadude                              0.035257
jordanrushworth                       0.003080
ts135992                              0.000597
Cinimod                               0.000575
nicmacd                               0.005310
not-logged-in-8eb7fc24412e2322f096    0.001982
syt18        

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())